In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

In [ ]:
data = gpd.read_file('data/seoul_kais_data/Z_KAIS_TL_SPRD_MANAGE_11000.shp', encoding='euc-kr')
data_area = gpd.read_file('data/seoul_kais_data/Z_KAIS_TL_KODIS_BAS_11000.shp', encoding='euc-kr')

In [ ]:
plt.rcParams['figure.figsize']=(10,20)

#데이터들 지도로 띄어보기
data.plot()
plt.show()

data_area.plot()
plt.show()

In [ ]:
data.head()

In [ ]:
def getRoadCrossAmount(mainRoad, roadList):
    crossAmount = 0

    for anotherRoad in roadList:
        if mainRoad.intersects(anotherRoad):
            crossAmount += 1
    

    return crossAmount

In [ ]:
def getCenterPosition(road):
    return road.centroid

In [ ]:
preprocessingRoadData = []

for area in (data_area['geometry']):
    roadList = data[area.crosses(data['geometry'])]

    preprocessingRoadList = roadList[(roadList.ROAD_LT < 2000) | (roadList.ROAD_BT < 8)]

    

    roadCenterPosList = preprocessingRoadList['geometry'].apply(getCenterPosition)
    xCenterPos = 0
    yCenterPos = 0
    for roadCenterPos in roadCenterPosList:
        xCenterPos += list(roadCenterPos.coords)[0][0]
        yCenterPos += list(roadCenterPos.coords)[0][1]
    


    if 22 >= len(preprocessingRoadList) >= 15:
        xCenterPos /= len(roadCenterPosList)
        yCenterPos /= len(roadCenterPosList)

        #preprocessingRoadList에 있는 도로들의 좌표를 모두 중심점으로 이동
        preprocessingRoadList['geometry'] = preprocessingRoadList['geometry'].translate(xoff=-xCenterPos, yoff=-yCenterPos)

        #preprocessingRoadList에 있는 도로들의 crossAmount를 구함
        preprocessingRoadList['crossAmount'] = preprocessingRoadList['geometry'].apply(getRoadCrossAmount, roadList=preprocessingRoadList['geometry'])

        preprocessingRoadData.append(preprocessingRoadList)


print("총 데이터 개수 : ", len(preprocessingRoadData))
    

In [ ]:
#도로의 폭, 길이, 교차로 개수 분포 그래프 출력 코드


roadWidthList = []
roadHeightList = []
roadCrossList = []

for area in (data_area['geometry']):
    roadList = data[area.crosses(data['geometry'])]
    roadAmount = len(roadList)

    roadWidth = 0
    roadHeight = 0
    roadCross = 0
    for roadIndex in roadList.index.to_list():
        checkRoad = roadList.loc[roadIndex]

        roadWidth = roadWidth + checkRoad['ROAD_BT']
        roadHeight = roadHeight + checkRoad['ROAD_LT']
        roadCross = roadCross + getRoadCrossAmount(checkRoad['geometry'], roadList['geometry'])

    if roadAmount > 0:
        roadWidth = roadWidth / roadAmount
        roadHeight = roadHeight / roadAmount
        roadCross = roadCross / roadAmount

        roadWidthList.append(roadWidth)
        roadHeightList.append(roadHeight)
        roadCrossList.append(roadCross)


sns.distplot(roadWidthList, kde=True, rug=True)
plt.title("Road Width Distribution")
plt.show()    

sns.distplot(roadHeightList, kde=True, rug=True)
plt.title("Road Height Distribution - Original Data")
plt.show()    

roadHeightList = list(filter(lambda x: x < 1500, roadHeightList))
sns.distplot(roadHeightList, kde=True, rug=True)
plt.title("Road Height Distribution - Preprocessing Data (<1500)")
plt.show()    

sns.distplot(roadCrossList, kde=True, rug=True)
plt.title("Road Cross Distribution")
plt.show()    

In [ ]:
road = data[data['RN']=='이태원로']
road.plot()

In [ ]:
def isContainPeopleInRoad(peoplePos, roadData):
    indexList = roadData.index.to_list()
    for roadIndex in indexList: #지도를 구성하는 도로 하나하나 확인
        roadWidth = roadData.loc[roadIndex]['ROAD_BT']

        posList = list((roadData.loc[roadIndex]['geometry']).coords)
        for posIndex in range(0, len(posList)): #도로의 좌표들을 확인
            if posList[posIndex] == posList[len(posList)-1]: #마지막 좌표는 확인하지 않음
                break
            
            firstPos = posList[posIndex]
            lastPos = posList[posIndex+1]
            

            #첫번째와 마지막 좌표와 도로폭으로 공간을 계산한 후 사람의 위치가 포함되는지 확인
            if firstPos[0] == lastPos[0]: #x좌표가 같은 경우
                if firstPos[0] - roadWidth/2 <= peoplePos[0] <= firstPos[0] + roadWidth/2:
                    if firstPos[1] <= peoplePos[1] <= lastPos[1] or firstPos[1] >= peoplePos[1] >= lastPos[1]:
                        return True
            elif firstPos[1] == lastPos[1]: #y좌표가 같은 경우
                if firstPos[1] - roadWidth/2 <= peoplePos[1] <= firstPos[1] + roadWidth/2:
                    if firstPos[0] <= peoplePos[0] <= lastPos[0] or firstPos[0] >= peoplePos[0] >= lastPos[0]:
                        return True
            else: #x좌표와 y좌표가 다른 경우
                if firstPos[0] - roadWidth/2 <= peoplePos[0] <= firstPos[0] + roadWidth/2:
                    if firstPos[1] - roadWidth/2 <= peoplePos[1] <= firstPos[1] + roadWidth/2:
                        return True
    
    return False